## Objective

Now that I know that my submodels perform a bit better than the model on the entire dataset, I can take advantage of the submodels and find the coefficients that are most associated with response variable.

In [1]:
CATEGORY_GROUPS_IN_QUESTION = [['Pick up Dead Animal', 'Animal Generic Request'],
['Abandoned Vehicles', 'Abandoned Bicycle'],
['Rodent Activity',	'Bed Bugs', 'Mice Infestation - Residential'],
['Sidewalk Repair', 'Sidewalk Repair (Make Safe)'],
['Needle Pickup'],
['Unsatisfactory Living Conditions', 'Poor Conditions of Property', 'Unsanitary Conditions - Establishment', 'Illegal Occupancy', 'Heat - ],Excessive  Insufficient'],
['Request for Pothole Repair'],
['Graffiti Removal']]

## Objective

Does this have statistically significant coefs, assuming homoskedacity and a linear predictor-response relationship and Normalized residuals and imperfect collinearity?

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
from tqdm import tqdm

from utilities import remove_one_feature

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))
from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type, adjusted_r2, transform_school, get_vifs

In [6]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [7]:
df_orig = transform_school(df_orig)
df_orig.shape

../preprocessing/helper_functions.py:58: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  df.school = df.school.str.extract(r'(\d\d?)').astype(int)


(516406, 40)

## Filterering by Abandoned vehicles

In [8]:
print CATEGORY_GROUPS_IN_QUESTION[1]
df_orig = df_orig[df_orig.TYPE.isin(CATEGORY_GROUPS_IN_QUESTION[1])]
df_orig.shape

['Abandoned Vehicles', 'Abandoned Bicycle']


(9541, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [9]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

../preprocessing/helper_functions.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  group[pd.np.abs(group - group.median()) > stds * group.std()] = pd.np.nan
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.where(-key, value, inplace=True)


(9390, 40)

## Remove `TYPE` col

In [10]:
df_outliers_removed.drop('TYPE', axis=1, inplace=True)

## Choosing columns

In [ ]:
['SubmittedPhoto',
 'race_asian',
 'race_other',
 'poverty_pop_below_poverty_level',
 'earned_income_per_capita',
 'poverty_pop_w_public_assistance',
 'poverty_pop_w_food_stamps',
 'poverty_pop_w_ssi',
 'school_std_dev',
 'housing_std_dev',
 'bedroom',
 'bedroom_std_dev',
 'rent_std_dev',
 'income_std_dev',
 'queue_wk',
 'Property_Type_Address',
 'Source_Citizens Connect App']

In [95]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'Property_Type', 'Source']
cols_census = [
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'housing',
]
cols_engineered = ['is_description']

In [96]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]
df.shape

(9390, 9)

## Removing NAs for cols like `school_std_dev`

In [97]:
aa = df.isnull().any().reset_index()
nas = aa[aa[0] == True]['index']
print nas

Series([], Name: index, dtype: object)


In [98]:
# this is a bad temporary band-aid
df = df.dropna(subset=nas.tolist())
df.shape

(9390, 9)

## Dummify

In [99]:
cols_to_dummify = [i for i in df.dtypes[df.dtypes == object].index if i != 'TYPE']
cols_to_dummify

['Property_Type', 'Source', 'housing']

In [100]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify, chosen_col_i=2)

Intersection is baseline 0 3
Self Service is baseline 1 3
own is baseline 2 3


In [70]:
df_dummified.shape

(9198, 11)

## Checking for multicollinearity

In [ ]:
df_dummified.head(1).T

In [18]:
get_vifs(df_dummified.drop(['SubmittedPhoto', 'is_description'], axis=1), 'COMPLETION_HOURS_LOG_10')

poverty_pop_w_food_stamps          4.325760
race_black                         3.214566
poverty_pop_below_poverty_level    2.800160
earned_income_per_capita           2.773845
queue_wk                           2.549613
queue_wk_open                      2.470314
Source_Citizens Connect App        2.341932
income                             2.298873
Source_Constituent Call            2.195432
race_hispanic                      2.052296
school                             2.029732
poverty_pop_w_ssi                  1.840288
housing_std_dev                    1.654134
school_std_dev                     1.605422
rent_std_dev                       1.561623
race_asian                         1.544201
income_std_dev                     1.535408
poverty_pop_w_public_assistance    1.520068
rent                               1.463006
value_std_dev                      1.438875
bedroom                            1.392077
value                              1.284623
bedroom_std_dev                 

## Running model

In [19]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import string
from StringIO import StringIO


In [20]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

## Use LassoCV to find col subsets

In [23]:
pipe = make_pipeline(StandardScaler(), LassoCV())
cv = ShuffleSplit(X_train.shape[0], n_iter=1, test_size=0.2, random_state=300)

In [26]:
params = {'lassocv__alphas': make_alphas(-2, -1)}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=cv, verbose=0)
model.fit(X_train, y_train)
pd.DataFrame(model.cv_results_).T.iloc[2:5]

,0,1,2
mean_test_score,0.0415425,0.0283155,-0.000141996
mean_train_score,0.0354414,0.0224905,0
param_lassocv__alphas,[0.01],[0.03],[0.1]


In [27]:
model.best_params_

{'lassocv__alphas': [0.01]}

In [29]:
'{} cols go to zero out of {}'.format(
    len(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0]),
    len(X_train.columns)
)

'17 cols go to zero out of 27'

In [30]:
cols_zero = list(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0])
cols_zero

['SubmittedPhoto',
 'race_asian',
 'race_other',
 'poverty_pop_below_poverty_level',
 'earned_income_per_capita',
 'poverty_pop_w_public_assistance',
 'poverty_pop_w_food_stamps',
 'poverty_pop_w_ssi',
 'school_std_dev',
 'housing_std_dev',
 'bedroom',
 'bedroom_std_dev',
 'rent_std_dev',
 'income_std_dev',
 'queue_wk',
 'Property_Type_Address',
 'Source_Citizens Connect App']

## Use subsetted cols to run lin reg

In [101]:
df_dummified.columns = [col.translate(None, string.punctuation).replace(' ', '') if col != 'COMPLETION_HOURS_LOG_10' else col for col in df_dummified.columns]

In [102]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop(['COMPLETION_HOURS_LOG_10'], axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [103]:
col_list = ' + '.join(df_dummified.drop(['COMPLETION_HOURS_LOG_10'], axis=1).columns)

est = smf.ols(
    'COMPLETION_HOURS_LOG_10 ~ {}'.format(col_list), 
    pd.concat([X_train, y_train], axis=1)).fit()

In [118]:
est.summary().tables[0]

Dep. Variable:,COMPLETION_HOURS_LOG_10,R-squared:,0.016
Model:,OLS,Adj. R-squared:,0.015
Method:,Least Squares,F-statistic:,15.34
Date:,"Fri, 24 Feb 2017",Prob (F-statistic):,1.45e-22
Time:,18:09:12,Log-Likelihood:,-3625.0
No. Observations:,7512,AIC:,7268.
Df Residuals:,7503,BIC:,7330.
Df Model:,8,,
Covariance Type:,nonrobust,,


### Getting adjusted $R^2$ on test set

In [104]:
y_pred = est.predict(X_test)

In [105]:
adjusted_r2(y_test, y_pred, num_features=X_test.shape[1])

0.010674602704032844

In [106]:
mean_squared_error(y_test, y_pred)**0.5

0.38797014008139136

In [119]:
y_test.describe()

count    1878.000000
mean        2.429112
std         0.390996
min         0.358252
25%         2.235064
50%         2.489926
75%         2.677769
max         3.857246
Name: COMPLETION_HOURS_LOG_10, dtype: float64

## Interpreting model

Which features are most associated with completion time?

In [114]:
df_results = pd.read_csv(StringIO(est.summary().tables[1].as_csv()), index_col=0).reset_index()
df_results.columns = ['coef_name'] + [i.rstrip().lstrip() for i in df_results.columns][1:]
df_results.coef_name = df_results.coef_name.map(lambda x: x.strip())
df_results = df_results.sort_values('P>|t|')
df_results['pct_diff_for_y'] = (10**df_results.coef - 1) * 100
df_results['pct_diff_for_y_abs'] = pd.np.abs((10**df_results.coef - 1) * 100)
df_results.sort_values('pct_diff_for_y_abs', inplace=True, ascending=False)
df_results.shape

(9, 8)

In [117]:
df_results[df_results['P>|t|'] < 0.1][df_results.coef_name != 'Intercept']

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],pct_diff_for_y,pct_diff_for_y_abs
4,racehispanic,0.1113,0.029,3.784,0.000,0.054 0.169,29.211152,29.211152
2,raceblack,0.1097,0.017,6.532,0.000,0.077 0.143,28.735997,28.735997
7,SourceCitizensConnectApp,0.0424,0.018,2.344,0.019,0.007 0.078,10.255433,10.255433
6,PropertyTypeAddress,0.0392,0.014,2.829,0.005,0.012 0.066,9.446027,9.446027
8,SourceConstituentCall,-0.0392,0.015,-2.678,0.007,-0.068 -0.011,-8.630763,8.630763
1,isdescription[T.True],-0.0345,0.013,-2.645,0.008,-0.060 -0.009,-7.636581,7.636581


In [109]:
df_dummified.COMPLETION_HOURS_LOG_10.map(lambda x: 10**x).describe()

count    9390.000000
mean      383.803573
std       405.297599
min         2.275000
25%       171.718333
50%       307.753611
75%       479.439375
max      7198.562222
Name: COMPLETION_HOURS_LOG_10, dtype: float64

In [94]:
scores = []

for col in X_train.columns:
    if col != 'Intercept':
        score = remove_one_feature([col], df_dummified)
        scores.append((col, score))
        
sorted(scores, key=lambda x: x[1])[::-1]        

[('raceother', 22.278),
 ('queuewkopen', 22.207),
 ('raceasian', 22.118),
 ('PropertyTypeAddress', 21.981),
 ('raceblack', 19.729),
 ('SourceCitizensConnectApp', 19.724),
 ('racehispanic', 19.227),
 ('isdescription', 17.195),
 ('SourceConstituentCall', 16.861)]

## Conclusion

- **Description** is associated with **better** completion time.
- **More Black or Hispanic people** in the area is associated with a **worse** completion time, each to the same degree, compared to White.
- From **mobile app** is associated with a **worse** completion time, compared to from website.
- From **call** is associated with a **better** completion time, compared to from website, to less of a degree than from mobile app.
- Location being **address** is associated with a **worse** completion time compared to intersection.

### Notable
- From **mobile app** is associated with a **worse** completion time, compared to from website.